In [1]:
from retraction_audit.audit import RetractionLookup
import os

os.environ["RETRACTION_WATCH_EMAIL"] = "james.sharpsteen@gmail.com"
os.environ["NCBI_AUTH_EMAIL"] = "james.sharpsteen@gmail.com"

lookup = RetractionLookup(True)

In [2]:
from wags_tails import CustomData
from wags_tails.utils.downloads import handle_zip, download_http
from wags_tails.utils.versioning import DATE_VERSION_PATTERN
from datetime import datetime, timezone

def _latest_version_cb() -> str:
    return datetime.now(tz=timezone.utc).strftime(DATE_VERSION_PATTERN)

def _download_cb(version, outfile_path) -> None:
        tqdm_params = {
            "disable": False,
            "unit": "B",
            "ncols": 80,
            "unit_divisor": 1024,
            "unit_scale": True,
        }
        download_http(
            "https://pmkb.weill.cornell.edu/therapies/download.csv",
            outfile_path,
            tqdm_params=tqdm_params,
        )

data_fetcher = CustomData(
    "pmkb",
    "csv",
    _latest_version_cb,
    _download_cb,
)
try:
    pmkb = data_fetcher.get_latest()
except:  # website isn't always up
    pmkb = data_fetcher.get_latest(from_local=True)

In [3]:
import polars as pl

df = pl.read_csv(pmkb[0])
unique_cites = set()
for row in df["Citations"].unique():
    for cite in row.split("|"):
        unique_cites.add(cite)
len(unique_cites)

In [29]:
for cite in unique_cites:
    match = re.match(pattern, cite)
    if not match:
        print(cite)

Cbioportal.org
10.1158/2159-8290
Kato H, Kato S, Kumabe T, Sonoda Y, Yoshimoto T, Kato S, Han SY, Suzuki T,
Kieran MW Targeted treatment for sonic hedgehog-dependent medulloblastoma. Neuro Oncol 2014;16(8):1037-47
Schittenhelm J Recent advances in subtyping tumors of the central nervous system using molecular data. Expert Rev Mol Diagn 2017;17(1):83-94
Neklason et al. BMC Cancer 2011, 11:424 Activating mutation in MET oncogene in familial colorectal cancer
Vellano CP, Schultz N, Karchin R, Ding L, Lu Y, Cheung LWT, Chen K, Shaw KR,
Bladder cancer (TCGA, 2017)
23689514
Spaggiari L, Mazzarol G, Viale G, Pece S, Di Fiore PP. Alterations of the Notch
Wong H, et al. Molecular targeted therapies in advanced gastric cancer: does tumor histology matter? Therap Adv Gastroenterol 2013;6(1):15-31
http://omim.org/entry/311770
induced epithelial-mesenchymal transition and cancer stem cell-like characteristics in
VH, Tran R, Soewito S, Minussi DC, Moreno D, Kong K, Dogruluk T, Lu H, Gao J,
OncoKB: h

In [ ]:
%%capture --no-stderr

from retraction_audit.citation_tools import SqlitePmidCache, get_pmid_from_author_title
from pathlib import Path
from tqdm import tqdm
import re

pmid_cache = SqlitePmidCache(Path("pmid_cache.db"))
pattern = r"(.*)\.[ ]?(.*)\..*"

pmids = []
for cite in tqdm(unique_cites):
    match = re.match(pattern, cite)
    if not match:
        continue
    author = match.groups()[0].split(" ")[0]
    title = match.groups()[1]
    try:
        pmid = get_pmid_from_author_title(author, title, pmid_cache)
    except:
        continue
    pmids.append(pmid)
print(len(pmids))

 24%|████████████████████▊                                                                   | 369/1562 [01:26<03:29,  5.69it/s]

In [2]:
retracted_ev = []
for pmid in pmids:
    retraction_record = lookup.get_retraction_by_pmid(str(pmid))
    if retraction_record:
        retracted_ev.append(retraction_record)

NameError: name 'pmids' is not defined

In [1]:
print(len(retracted_ev))
from tools import store_results
store_results("PMKB", len(retracted_ev), retracted_ev, len(unique_cites))

NameError: name 'retracted_ev' is not defined